![](./Resources/ai-sk-add-wechat.png)

# SK 日志集成
为了便于理解SK 的调用细节，因此开启日志是必不可少，这一节就来具体展开说明下具体如何使用`NLog`和自定义`IHttpClientLogger`来记录SK日志。

## 安装 NLog NuGet包

In [40]:
#r "nuget:NLog.Extensions.Logging"

Installed Packages NLog.Extensions.Logging, 5.3.14

## 添加扩展方法
添加`Config/NLogServiceCollectionExtensions.cs`，代码如下：

In [37]:
using Microsoft.Extensions.DependencyInjection;
using NLog.Extensions.Logging;
using NLog.Targets;

public static IServiceCollection AddNLogging(this IServiceCollection services)
{
    // 定义文件日志输出目标
    var fileTarget = new FileTarget()
    {
        FileName = "sk-demo.log", AutoFlush = true, DeleteOldFileOnStartup = true
    };
    // 定义控制台日志输出目标
    var consoleTarget = new ConsoleTarget();
    
    var config = new NLog.Config.LoggingConfiguration();
    // 定义日志输出规则(输出所有Trace级别及以上的日志到控制台)
    config.AddRule(
        NLog.LogLevel.Trace,
        NLog.LogLevel.Fatal,
        target: fileTarget,  // 这里采用文件输出
        "*");// * 表示所有Logger
    // 注册NLog
    services.AddLogging(loggingBuilder => loggingBuilder.AddNLog(config));
    return services;
}

## 日志服务注册并测试

修改`Config/AiProviderRegister.cs`类的`BuildKernel`方法，注册日志服务，修改如下，主要是在`BuildKernel` 方法中添加`builder.Services.AddNLogging();`，此为启用`Microsoft.SemanticKernel*`日志。
```
public abstract class AiProviderRegister
{
    // 省略其他代码    
    public virtual Kernel BuildKernel(IServiceProvider serviceProvider, AiProvider aiProvider)
    {
        // 创建Kernel构建器
        var builder = Kernel.CreateBuilder();

        // 注册日志服务
        builder.Services.AddNLogging();

        // 省略其他代码
    }
}    

```

紧接着安装相关的NuGet包并引入之前的抽象定义的类文件：

In [41]:
#r "nuget:Microsoft.Extensions.Configuration"
#r "nuget:Microsoft.Extensions.Configuration.Json"
#r "nuget:Microsoft.Extensions.Configuration.Binder"
#r "nuget:Microsoft.SemanticKernel"
#r "nuget:Microsoft.SemanticKernel.Connectors.OpenAI"
#r "nuget:Microsoft.SemanticKernel.Connectors.AzureOpenAI"

#!import Config/AiProvider.cs
#!import Config/AiOptions.cs
#!import Config/AiSettings.cs
#!import Config/AiProviderRegister.cs
#!import Config/AzureOpenAiRegister.cs
#!import Config/OpenAiRegister.cs
#!import Config/OpenAiCompatibleAiRegister.cs
#!import Config/AiProviderRegisterFactory.cs
#!import Config/NLogServiceCollectionExtensions.cs
#!import Config/KernelServiceCollectionExtensions.cs

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Binder, 8.0.2 Microsoft.Extensions.Configuration.Json, 8.0.1 Microsoft.SemanticKernel, 1.25.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.25.0 Microsoft.SemanticKernel.Connectors.OpenAI, 1.25.0

以下为测试代码：

In [42]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

using PolyglotKernel = Microsoft.DotNet.Interactive.Kernel;// 引入交互式的内核命名空间，以便用户输入

// 创建DI 容器
var services = new ServiceCollection();

// 注册全局日志服务
services.AddNLogging();

// 注册所有内核服务
services.RegisterKernels();

// 构建ServiceProvider
var serviceProvider = services.BuildServiceProvider();
var aiProviderCode = await PolyglotKernel.GetInputAsync("请输入AI服务提供商编码：");

// 获取指定的Kernel服务
var kernel = serviceProvider.GetKeyedService<Kernel>(aiProviderCode);
var response = await kernel.InvokePromptAsync("一句话简单介绍ML.NET。");
response.Display();

ML.NET是一个基于.NET平台的开源机器学习框架，允许开发者在他们的应用程序中构建、训练和部署自定义的机器学习模型。 Function InvokePromptAsync_6f68aec2902947fa9bc893656f2e2606 Name InvokePromptAsync_6f68aec2902947fa9bc893656f2e2606 PluginName <null> Description Generic function, unknown purpose Metadata Microsoft.SemanticKernel.KernelFunctionMetadata Name InvokePromptAsync_6f68aec2902947fa9bc893656f2e2606 PluginName <null> Description Generic function, unknown purpose Parameters (empty) ReturnParameter Microsoft.SemanticKernel.KernelReturnParameterMetadata Description ParameterType <null> Schema <null> AdditionalProperties (empty) ExecutionSettings (empty) Metadata key type value Id System.String chatcmpl-AOKahqLY6xFWwl20vRz8OwSsGq4DN CreatedAt System.DateTimeOffset 2024-10-31 08:13:07Z SystemFingerprint System.String fp_d54531d9eb Usage OpenAI.Chat.ChatTokenUsage OpenAI.Chat.ChatTokenUsage OutputTokenCount 38 InputTokenCount 14 TotalTokenCount 52 OutputTokenDetails <null> Refusal <null> <null> FinishReason System.String Stop ContentTokenLogProbabilities OpenAI.ChangeTrackingList<OpenAI.Chat.ChatTokenLogProbabilityDetails> Culture Parent Parent Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mon, Tue, Wed, Thu, Fri, Sat ] ShortestDayNames [ Su, Mo, Tu, We, Th, Fr, Sa ] DayNames [ Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday ] AbbreviatedMonthNames [ Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec, ] MonthN

这时可以在文件系统搜索上面指定的文件`sk-demo.log`文件，可以看到类似如下的日志输出：
```
2024-10-31 15:41:07.0746|INFO|Microsoft.SemanticKernel.KernelFunction|Function InvokePromptAsync_e4f527d8548f4e3883389604a34a819a invoking.
2024-10-31 15:41:07.0951|TRACE|Microsoft.SemanticKernel.KernelFunction|Function arguments: {}
2024-10-31 15:41:07.0951|TRACE|Microsoft.SemanticKernel.KernelFunctionFactory|Rendered prompt: 一句话简单介绍ML.NET。
2024-10-31 15:41:07.1145|TRACE|Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService|ChatHistory: [{"Role":{"Label":"user"},"Items":[{"$type":"TextContent","Text":"\u4E00\u53E5\u8BDD\u7B80\u5355\u4ECB\u7ECDML.NET\u3002"}]}], Settings: null
2024-10-31 15:41:07.8757|INFO|Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService|Prompt tokens: 14. Completion tokens: 39. Total tokens: 53.
2024-10-31 15:41:07.8757|INFO|Microsoft.SemanticKernel.KernelFunction|Function InvokePromptAsync_e4f527d8548f4e3883389604a34a819a succeeded.
2024-10-31 15:41:07.8757|TRACE|Microsoft.SemanticKernel.KernelFunction|Function result: ML.NET是一个由微软开发的开源跨平台机器学习框架，旨在让.NET开发者能够方便地将机器学习模型集成到他们的应用程序中。
2024-10-31 15:41:07.8757|INFO|Microsoft.SemanticKernel.KernelFunction|Function completed. Duration: 0.7831405s

```
>Tips: Windows 用户请安装`Everything（https://www.voidtools.com）` 搜索`sk-demo.log`即可，我的文件目录在(C:\Users\shengjie\.nuget\packages\microsoft.dotnet-interactive\1.0.522904\tools\net9.0\any)。